# Multi-Analyst Text Analysis Pipeline

In this notebook we demonstrate the full pipeline for analyzing text through multiple specialist lenses (rhetorician, syntactician, lexicologist, etc.) and synthesizing their observations into 
* a unified writing style analysis from the text samples,
* an instruction of how to write in the analyzed style.

The workflow is *agentic* in that it involves several distinct agents built on Large Language Models (LLMs), however, the order and relation between each agent is set, not dynamically derived.

The analysis and its details are described in this blog post: **INSERT LINK**

## Installations and Preparations
External libraries are installed and tested to be in working order.

Key dependencies:
* LiteLLM (model router such that different LLM APIs can be readily employed)
* Jinja (create prompts with variables and conditional logic)
* Pydantic (create prompts from variables with validation)

**Install requirements.** Only needed if running in fresh kernel.

In [1]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.1.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


Check that LiteLLM was installed correctly. List the providers available via LiteLLM router.

In [2]:
try:
    import litellm
    print('Providers\n=========')
    print('* ' + '\n* '.join(litellm.LITELLM_CHAT_PROVIDERS))
except ImportError as e:
    print(f"✗ Cannot import litellm: {e}")

Providers
* openai
* openai_like
* bytez
* xai
* custom_openai
* text-completion-openai
* cohere
* cohere_chat
* clarifai
* anthropic
* anthropic_text
* replicate
* huggingface
* together_ai
* datarobot
* openrouter
* cometapi
* vertex_ai
* vertex_ai_beta
* gemini
* ai21
* baseten
* azure
* azure_text
* azure_ai
* sagemaker
* sagemaker_chat
* bedrock
* vllm
* nlp_cloud
* petals
* oobabooga
* ollama
* ollama_chat
* deepinfra
* perplexity
* mistral
* groq
* nvidia_nim
* cerebras
* baseten
* ai21_chat
* volcengine
* codestral
* text-completion-codestral
* deepseek
* sambanova
* maritalk
* cloudflare
* fireworks_ai
* friendliai
* watsonx
* watsonx_text
* triton
* predibase
* databricks
* empower
* github
* custom
* litellm_proxy
* hosted_vllm
* llamafile
* lm_studio
* galadriel
* gradient_ai
* github_copilot
* novita
* meta_llama
* featherless_ai
* nscale
* nebius
* dashscope
* moonshot
* v0
* heroku
* oci
* morph
* lambda_ai
* vercel_ai_gateway
* wandb
* ovhcloud
* lemonade


## Initialize Base Objects
The base objects part of the current project library (`belletrist`) are initialized. They are:
* `LLM`: the LLM object.
* `LLMConfig`: the configuration of the LLM object, such as what model to use.
* `PromptMaker`: generates prompts from templates and variables
* `DataSampler`: retrieves and samples text at a source directory
* `ResultStore`: simple database object to save intermediate and final outputs

The LLM to use is set by the `model_string`, which is constructed as `<provider>/<model>`, the providers defined by the `litellm` package, see in particular `litellm.LITELLM_CHAT_PROVIDERS`. The API key to the provider should be stored in an environment variable with name defined in `model_provider_api_key_env_var`. You need to create that yourself for the provider of interest. 

Do **not** store the API key as a string variable directly in the notebook, you're at risk of exposing it.

In [3]:
model_string = 'together_ai/Qwen/Qwen3-235B-A22B-Instruct-2507-tput'
model_provider_api_key_env_var = 'TOGETHER_AI_API_KEY'

In [4]:
import os
from pathlib import Path
from belletrist import LLM, LLMConfig, PromptMaker, DataSampler, ResultStore

llm = LLM(LLMConfig(
    model=model_string,
    api_key=os.environ.get(model_provider_api_key_env_var)
))
prompt_maker = PromptMaker()
sampler = DataSampler(
    data_path=(Path(os.getcwd()) / "data" / "russell").resolve()
)
store = ResultStore(Path(f"{os.getcwd()}/belletrist_storage.db"))

In case a clean run is wanted, the old contents of the database are discarded with a result store reset. Do **not** run this reset if content should be preserved from previous runs. 

In [5]:
store.reset()

## Generate and Store Text Samples to be Analyzed

The `DataSampler` retrieves paragraphs from the corpus of text. The retrieval can be a random sample of consecutive paragraphs (via the method `sample_segment`) or a specific file and paragraph range (via the method `get_paragraph_chunk`).

As illustration of the process, a random four-paragraph long segment is sampled below.

In [6]:
text_sample = sampler.sample_segment(p_length=4)
print(f'Text source: {text_sample.file_path}')
print(f'Paragraph range: {text_sample.paragraph_start} - {text_sample.paragraph_end}')
print(f'\n{text_sample.text}')

Text source: /Users/andersohrn/PycharmProjects/russell_writes/data/russell/education_and_the_good_life.txt
Paragraph range: 88 - 92

Questions of physical health, strictly speaking, lie outside the scope
of this book, and must be left to medical practitioners. I shall touch
on them only where they have psychological importance. But physical and
mental are scarcely distinguishable in the first year of life. Moreover
the educator in later years may find himself handicapped by purely
physiological mistakes in handling the infant. We cannot therefore
altogether avoid trespassing upon ground which does not of right belong
to us.

The new-born infant has reflexes and instincts, but no habits. Whatever
habits it may have acquired in the womb are useless in its new
situation: even breathing sometimes has to be taught, and some children
die because they do not learn the lesson quickly enough. There is one
well-developed instinct, the instinct of sucking; when the child is
engaged in this occupa

A number of text samples are retrieved (**set `n_sample` to the desired number**) and each sample comprises a set number of paragraphs (**set `m_paragraphs_per_sample` to the desired number**). These text samples are stored in the project result stored with keys like `sample_001`, `sample_002` and so on; these keys are henceforth referring to specific text samples.

In [7]:
print(sampler.get_paragraph_chunk(2, slice(10,15)).text)

Thus it becomes evident that the real table, if there is one, is not the
same as what we immediately experience by sight or touch or hearing. The
real table, if there is one, is not _immediately_ known to us at all,
but must be an inference from what is immediately known. Hence, two very
difficult questions at once arise; namely, (1) Is there a real table at
all? (2) If so, what sort of object can it be?

It will help us in considering these questions to have a few simple
terms of which the meaning is definite and clear. Let us give the name
of 'sense-data' to the things that are immediately known in sensation:
such things as colours, sounds, smells, hardnesses, roughnesses, and
so on. We shall give the name 'sensation' to the experience of being
immediately aware of these things. Thus, whenever we see a colour,
we have a sensation _of_ the colour, but the colour itself is a
sense-datum, not a sensation. The colour is that _of_ which we are
immediately aware, and the awareness itself i

In [8]:
n_sample = 5
m_paragraphs_per_sample = 5

for _ in range(n_sample):
    sample_id = f'sample_{len(store.list_samples()) + 1:03d}'
    segment = sampler.sample_segment(p_length=m_paragraphs_per_sample)
    store.save_segment(sample_id, segment)

In [9]:
print('Sample keys:\n============')
store.list_samples()

Sample keys:


['sample_001', 'sample_002', 'sample_003', 'sample_004', 'sample_005']

## Step 1: Construct the Analyst Agents and Analyze Text Samples

Send the text samples through each specialist analyst. Each produces an independent analysis from their domain expertise.

**Prompt structure for each analyst is:**
1. Preamble instruction of task ahead 
2. Analyst-specific instruction template
3. Text to analyze

Note that the execution of this can take time since it involves invoking LLMs, once per analyst and text sample. These are however independent analyses, and can therefore *in principle* be run in parallel, though the implementation below does not utilize that fact.

Note that the agents are distinguished by their prompts, which are obtained via prompt models defined within the `bellatrist` project library.

In [10]:
from belletrist.models import (
    PreambleInstructionConfig,
    PreambleTextConfig,
    RhetoricianConfig,
    SyntacticianConfig,
    LexicologistConfig,
    InformationArchitectConfig,
    EfficiencyAuditorConfig,
    CrossPerspectiveIntegratorConfig,
)
ANALYSTS = ["rhetorician", "syntactician", "lexicologist", "information_architect", "efficiency_auditor"]
ANALYST_CONFIGS = {
    "rhetorician": RhetoricianConfig,
    "syntactician": SyntacticianConfig,
    "lexicologist": LexicologistConfig,
    "information_architect": InformationArchitectConfig,
    "efficiency_auditor": EfficiencyAuditorConfig,
}

def build_analyst_prompt(preamble_instruction: str, analyst_prompt: str, preamble_text: str) -> str:
    """
    Helper function to construct the full prompt for an analyst.
    
    """
    return f"{preamble_instruction}\n\n{analyst_prompt}\n\n{preamble_text}"

In [11]:
# Get all samples from the store
all_samples = store.list_samples()
print(f"Processing {len(all_samples)} samples with {len(ANALYSTS)} analysts each\n")

# Outer loop: iterate over each text sample
for sample_id in all_samples:
    print(f"Sample: {sample_id}")
    
    # Get the sample text
    sample = store.get_sample(sample_id)
    text = sample['text']
    
    # Build shared prompt components (reused across all analysts for this sample)
    preamble_instruction = prompt_maker.render(PreambleInstructionConfig())
    preamble_text = prompt_maker.render(PreambleTextConfig(text_to_analyze=text))
    
    # Inner loop: run each analyst on this sample
    for analyst_name in ANALYSTS:
        print(f"  Running {analyst_name}...", end=" ")
        
        # Get analyst-specific prompt using the config class
        analyst_config = ANALYST_CONFIGS[analyst_name]()
        analyst_prompt = prompt_maker.render(analyst_config)
        full_prompt = build_analyst_prompt(preamble_instruction, analyst_prompt, preamble_text)
        
        # Run analysis and save result
        response = llm.complete(full_prompt)
        store.save_analysis(sample_id, analyst_name, response.content, response.model)
        
        print(f"✓ ({len(response.content)} chars)")
    
    print()

print(f"All analyses complete for {len(all_samples)} samples")

Processing 5 samples with 5 analysts each

Sample: sample_001
  Running rhetorician... ✓ (9492 chars)
  Running syntactician... ✓ (8694 chars)
  Running lexicologist... ✓ (8726 chars)
  Running information_architect... ✓ (8214 chars)
  Running efficiency_auditor... ✓ (8800 chars)

Sample: sample_002
  Running rhetorician... ✓ (9531 chars)
  Running syntactician... ✓ (8197 chars)
  Running lexicologist... ✓ (8874 chars)
  Running information_architect... ✓ (8900 chars)
  Running efficiency_auditor... ✓ (8750 chars)

Sample: sample_003
  Running rhetorician... ✓ (9846 chars)
  Running syntactician... ✓ (8630 chars)
  Running lexicologist... ✓ (9161 chars)
  Running information_architect... ✓ (8655 chars)
  Running efficiency_auditor... ✓ (9000 chars)

Sample: sample_004
  Running rhetorician... ✓ (9749 chars)
  Running syntactician... ✓ (8803 chars)
  Running lexicologist... ✓ (9513 chars)
  Running information_architect... ✓ (8599 chars)
  Running efficiency_auditor... ✓ (9449 chars)

S

Verification that analysis was run as expected and yielded analysis results. Excerpt of one specific analysis retrieved from project database and printed for illustration.

In [12]:
sample_id = 'sample_001'
is_complete = store.is_complete(sample_id, ANALYSTS)
print(f"Analysis complete: {is_complete}")

# Retrieve sample and all analyses (both are now dicts)
sample, analyses = store.get_sample_with_analyses(sample_id)

print(f"\nSample: {sample['sample_id']}")
print(f"Source: File {sample['file_index']}, paragraphs {sample['paragraph_start']}-{sample['paragraph_end']}")
print(f"Analyses available: {list(analyses.keys())}")

# Examine one analysis
print(f"\n--- Rhetorician Output (first 500 chars) ---")
print(analyses.get("rhetorician", "Not found")[:500])

Analysis complete: True

Sample: sample_001
Source: File 2, paragraphs 32-37
Analyses available: ['efficiency_auditor', 'information_architect', 'lexicologist', 'rhetorician', 'syntactician']

--- Rhetorician Output (first 500 chars) ---
### RHETORICAL STRATEGY AND STANCE ANALYSIS  
**Text Segment:** Philosophical prose on the existence of an external world and other minds.

---

#### 1. **WRITER'S POSITION**

- **Persona:** The writer adopts an **authoritative yet measured** persona—calm, reflective, and rigorously logical. The tone is that of a **philosopher as guide**, not a polemicist. There is a clear commitment to intellectual honesty and precision.

  > Example: “In one sense it must be admitted that we can never prove th


## Step 2a: Pattern Recognition (Cross-Perspective Integration) per Text Sample

Synthesize all analyst perspectives to identify interactions, tensions, and load-bearing features. This is a per-text-cross-analyst transformation. This looks to integrate multiple perspectives on each text sample and indirectly distill the information content in the assessments of the text samples. 

If only a subset of samples are to be analyzed, filter or slice the list `samples_to_analyze`, which is a list of sample IDs, as created above.

In [13]:
samples_to_analyze = store.list_samples()
print(samples_to_analyze)

['sample_001', 'sample_002', 'sample_003', 'sample_004', 'sample_005']


In [14]:
def build_pattern_prompt_from_(text: str, analyses: dict):
    """Convenience function to create the prompt, since the prompt depends on the kinds of analysts to integrate.
    
    """
    sample, analyses = store.get_sample_with_analyses(sample_id)

    analyst_info = {}
    for analyst_name in ANALYSTS:
        config_class = ANALYST_CONFIGS[analyst_name]
        analyst_info[analyst_name] = {
            'analysis': analyses[analyst_name],
            'analyst_descr_short': config_class.description()
        }

    pattern_config = CrossPerspectiveIntegratorConfig(
        original_text=sample['text'],
        analysts=analyst_info
    )
    return prompt_maker.render(pattern_config)

Note that this loop can take time to execute, since LLMs are called. Each analysis is independent and can therefore in principle be parallelized, though the implementation below does not do that.

Note also that the cross-perspective per-text result are stored in the result store, keyed on the sample ID and the analyst kind.

In [15]:
for sample_id in samples_to_analyze:
    print(f"Running Cross-Perspective Integrator agent for {sample_id}...", end=" ")
    
    sample, analyses = store.get_sample_with_analyses(sample_id)
    pattern_prompt = build_pattern_prompt_from_(text=sample['text'], analyses=analyses)

    pattern_response = llm.complete(pattern_prompt)
    
    # Store pattern recognition result in result store
    store.save_analysis(
        sample_id, 
        CrossPerspectiveIntegratorConfig.analyst_name(), 
        pattern_response.content, 
        pattern_response.model
    )
    
    print(f"✓ ({len(pattern_response.content)} chars)")

Running Cross-Perspective Integrator agent for sample_001... ✓ (8788 chars)
Running Cross-Perspective Integrator agent for sample_002... ✓ (8812 chars)
Running Cross-Perspective Integrator agent for sample_003... ✓ (9405 chars)
Running Cross-Perspective Integrator agent for sample_004... ✓ (9246 chars)
Running Cross-Perspective Integrator agent for sample_005... ✓ (9321 chars)


In [16]:
sample_id = 'sample_005'
sample, analyses = store.get_sample_with_analyses(sample_id)
print(analyses.keys())

print(f"\n--- Pattern Analyst Output (first 2000 chars) ---")
print(analyses.get("cross_perspective_integrator", "Not found")[:2000])

dict_keys(['cross_perspective_integrator', 'efficiency_auditor', 'information_architect', 'lexicologist', 'rhetorician', 'syntactician'])

--- Pattern Analyst Output (first 2000 chars) ---
# **CROSS-PERSPECTIVE INTEGRATION ANALYSIS**

---

## **I. EXTRACTED TECHNIQUES**

---

### **1. Name: Core Claim Anchoring**  
**Specification:** Place the main philosophical assertion in the first clause of the paragraph as a definitional or normative claim, using a copular structure ("X is Y") to establish an axiomatic premise. Follow immediately with elaboration, consequence, or contrast.  
**Example from text:**  
> "The good which it concerns us to remember is the good which it lies in our power to create--the good in our own lives and in our attitude towards the world."  
**Source observations:**  
- *Information_architect*: "Topic sentence: Explicit and initial... sets the moral-psychological frame."  
- *Rhetorician*: "Foundational ethical-psychological claim presented axiomatically."  
- *S

## Stage 2b: Cross-Text Synthesis of Integrated Analyses

Patterns that appear across multiple text analyses are synthesized. This stage takes all the cross-perspective integration outputs and identifies overaching techniques, complementary findings, and so on, in order to construct a highly specific conclusion on the techniques that are employed in the text samples. It attempts in other words a synthesis of all analysis, across perspectives and across text samples.

This is a single document. In order to track provenance, the text samples and analyst types that went into its construction are gathered and included in the storage in the project database.

In [17]:
from belletrist.models import CrossTextSynthesizerConfig

# Get all samples that have cross-perspective integration results
all_samples = store.list_samples()
pattern_analyst = CrossPerspectiveIntegratorConfig.analyst_name()

# Retrieve all pattern recognition analyses
integrated_analyses = {}
for sample_id in all_samples:
    pattern_analysis = store.get_analysis(sample_id, pattern_analyst)
    if pattern_analysis:
        integrated_analyses[sample_id] = pattern_analysis
    else:
        print(f"⚠ Sample {sample_id} missing cross-perspective integration results")

print(f"Found {len(integrated_analyses)} cross-perspective integration results")
print(f"Sample IDs: {list(integrated_analyses.keys())}")

if len(integrated_analyses) < 2:
    print(f"\n⚠ Need at least 2 pattern recognition analyses for cross-text synthesis. Got {len(integrated_analyses)}.")

Found 5 cross-perspective integration results
Sample IDs: ['sample_001', 'sample_002', 'sample_003', 'sample_004', 'sample_005']


This is where the analysis is run. This can take time, since it requires running an LLM, however, only one LLM call in total.

In [18]:
cross_text_config = CrossTextSynthesizerConfig(
    integrated_analyses=integrated_analyses
)
cross_text_prompt = prompt_maker.render(cross_text_config)
    
print("Running Cross-Text Synthesis...", end=" ")
cross_text_response = llm.complete(cross_text_prompt)
print(f"✓ ({len(cross_text_response.content)} chars)")
    
# Save to ResultStore with auto-generated ID and full provenance
sample_contributions = [(sid, pattern_analyst) for sid in integrated_analyses.keys()]
cross_text_id = store.save_synthesis(
    synthesis_type=cross_text_config.synthesis_type(),
    output=cross_text_response.content,
    model=cross_text_response.model,
    sample_contributions=sample_contributions,
    config=cross_text_config
)
print(f"Saved as: {cross_text_id}")

Running Cross-Text Synthesis... ✓ (8424 chars)
Saved as: cross_text_synthesis_001


In [19]:
#  # Clean up orphaned records caused by disabled foreign keys
#  cursor = store.conn.execute("""
#      DELETE FROM synthesis_samples
#      WHERE synthesis_id NOT IN (SELECT synthesis_id FROM syntheses)
#  """)
#  print(f"Deleted {cursor.rowcount} orphaned synthesis_samples records")#
#
#  cursor = store.conn.execute("""
#      DELETE FROM synthesis_metadata
#      WHERE synthesis_id NOT IN (SELECT synthesis_id FROM syntheses)
#  """)
#  print(f"Deleted {cursor.rowcount} orphaned synthesis_metadata records")#
#
#  store.conn.commit()
#
#  # Verify cleanup
#  orphans = store.conn.execute("""
#      SELECT COUNT(*) FROM synthesis_samples ss
#      LEFT JOIN syntheses s ON ss.synthesis_id = s.synthesis_id
#      WHERE s.synthesis_id IS NULL
#  """).fetchone()[0]
#  print(f"Remaining orphaned records: {orphans}")

In [20]:
#sample_contributions = [(sid, pattern_analyst) for sid in integrated_analyses.keys()]
#cross_text_id = store.save_synthesis(
#    synthesis_type=cross_text_config.synthesis_type(),
#    output=cross_text_response.content,
#    model=cross_text_response.model,
#    sample_contributions=sample_contributions,
#    config=cross_text_config
#)
#print(f"Saved as: {cross_text_id}")

In [21]:
print("\n--- Cross-Text Synthesis (first 1000 chars) ---")
print(cross_text_response.content[:2000])


--- Cross-Text Synthesis (first 1000 chars) ---
# **CROSS-TEXT SYNTHESIS: PROVISIONAL PRINCIPLES FOR PHILOSOPHICAL PROSE**

---

## **I. STABLE CORE PATTERNS**  
*(Techniques appearing in ≥4 of 5 texts)*

---

### **1. Main-Claim-First Clause Anchoring**  
**Frequency**: 5/5  
**Description**: Core assertions are placed in independent clauses at or near the beginning of sentences. When subordination occurs, the main clause (containing the primary claim) precedes or immediately follows the subordinate clause.  
**Mechanical Specs**:  
- Main clause: 6–15 words  
- Subordinate clause: 15–35 words  
- Core claim never embedded; always in main or initial clause  
**Examples**:  
> "Other people are represented to me by certain sense-data..."  
> "Reason is a harmonising, controlling force rather than a creative one."  
> "The first characteristic of two appearances... is continuity."  
> "We can now begin to understand one of the fundamental differences between physics and psychology."  


## Stage 3: Synthesize Prescriptive Writing Document

The final stage converts the descriptive cross-text synthesis into actionable prescriptive writing principles. This generates a style guide that can be used to instruct an LLM to write in a similar style.

In [22]:
cross_text_syntheses = store.list_syntheses('cross_text_synthesis')
cross_text_synthesis_to_analyze_id = cross_text_syntheses[-1]
print(f"Using cross-text synthesis: {cross_text_synthesis_to_analyze_id}")

Using cross-text synthesis: cross_text_synthesis_001


The following step executes the LLM and can therefore take time. The result is stored for provenance tracking in the project database alongside relevant metadata.

In [23]:
from belletrist.models import SynthesizerOfPrinciplesConfig

# Build principles guide config and prompt
cross_text_synthesis_to_analyze = store.get_synthesis(cross_text_synthesis_to_analyze_id)
principles_config = SynthesizerOfPrinciplesConfig(
    synthesis_document=cross_text_synthesis_to_analyze['output']
)
principles_prompt = prompt_maker.render(principles_config)
    
# Run principles synthesis
print("Running Synthesizer of Writing Style Principles...", end=" ")
principles_response = llm.complete(principles_prompt)
print(f"✓ ({len(principles_response.content)} chars)")
    
# Save to ResultStore with parent linkage (inherits provenance)
principles_id = store.save_synthesis(
    synthesis_type=principles_config.synthesis_type(),
    output=principles_response.content,
    model=principles_response.model,
    sample_contributions=[],  # Inherited from parent
    config=principles_config,
    parent_synthesis_id=cross_text_synthesis_to_analyze_id
)
print(f"Saved as: {principles_id}")

Running Synthesizer of Writing Style Principles... ✓ (9632 chars)
Saved as: principles_guide_001


In [24]:
print("\n--- Principles Guide (first 2000 chars) ---")
print(principles_response.content[:2000])


--- Principles Guide (first 2000 chars) ---
# A GUIDE TO PHILOSOPHICAL CLARITY  
## Principles Extracted from Pattern Analysis

---

### PART I: FOUNDATIONS  
## **Core Principles**

---

**1. Anchor every claim in a simple, early main clause.**  
Place your core assertion in a grammatically independent clause of 6–15 words, positioned at or near the beginning of the sentence. Do not bury the claim inside subordinate constructions or after lengthy qualifications. This creates immediate cognitive grounding, allowing the reader to process complexity *in relation to* a stable proposition.  
> "Other people are represented to me by certain sense-data..."  
> "Reason is a harmonising, controlling force rather than a creative one."  
> "The first characteristic of two appearances... is continuity."  
When qualifications are necessary, subordinate them—do not let them precede or enclose the main claim. This structure prevents ambiguity and maintains epistemic authority.  
**Dependencies**: E

## Querying Synthesis Metadata

The ResultStore tracks full provenance for all syntheses. Query metadata to understand what samples, analysts, and models contributed to each synthesis.

**This code is for convenience and not required to generate the writing instruction.**

In [25]:
# List all syntheses
print("All Syntheses")
print("=" * 50)
for synth_type in ['cross_text_synthesis', 'principles_guide']:
    syntheses = store.list_syntheses(synth_type)
    print(f"\n{synth_type}: {len(syntheses)} found")
    for synth_id in syntheses:
        print(f"  - {synth_id}")

# Get detailed metadata for a synthesis
if store.list_syntheses():
    print("\n\nDetailed Metadata Example")
    print("=" * 50)
    
    # Get first principles guide (or first cross-text if none)
    principles = store.list_syntheses('principles_guide')
    if principles:
        synth_id = principles[0]
    else:
        synth_id = store.list_syntheses()[0]
    
    synth_with_meta = store.get_synthesis_with_metadata(synth_id)
    
    print(f"\nSynthesis ID: {synth_with_meta['synthesis_id']}")
    print(f"Type: {synth_with_meta['type']}")
    print(f"Model: {synth_with_meta['model']}")
    print(f"Created: {synth_with_meta['created_at']}")
    print(f"Parent: {synth_with_meta['parent_id']}")
    
    if synth_with_meta.get('metadata'):
        meta = synth_with_meta['metadata']
        print(f"\nMetadata:")
        print(f"  Samples: {meta['num_samples']}")
        print(f"  Sample IDs: {meta['sample_ids']}")
        print(f"  Model homogeneous: {meta['is_homogeneous_model']}")
        print(f"  Models used: {meta['models_used']}")

# Get full provenance tree
if store.list_syntheses('principles_guide'):
    print("\n\nFull Provenance Tree")
    print("=" * 50)
    
    for p_id in store.list_syntheses('principles_guide'):
        provenance = store.get_synthesis_provenance(p_id)
    
        print(f"\nPrinciples Guide: {provenance['synthesis']['synthesis_id']}")
        print(f"  Created: {provenance['synthesis']['created_at']}")
        print(f"  Model: {provenance['synthesis']['model']}")
    
        if provenance['parent']:
            parent = provenance['parent']
            print(f"\n  Parent (Cross-Text): {parent['synthesis']['synthesis_id']}")
            print(f"    Sample contributions: {len(parent['sample_contributions'])}")
            for sample_id, analyst in parent['sample_contributions'][:3]:
                print(f"      - {sample_id} / {analyst}")
            if len(parent['sample_contributions']) > 3:
                print(f"      ... and {len(parent['sample_contributions']) - 3} more")

All Syntheses

cross_text_synthesis: 1 found
  - cross_text_synthesis_001

principles_guide: 1 found
  - principles_guide_001


Detailed Metadata Example

Synthesis ID: principles_guide_001
Type: principles_guide
Model: together_ai/Qwen/Qwen3-235B-A22B-Instruct-2507-tput
Created: 2025-11-25T18:08:35.705147
Parent: cross_text_synthesis_001

Metadata:
  Samples: 0
  Sample IDs: []
  Model homogeneous: True
  Models used: ['together_ai/Qwen/Qwen3-235B-A22B-Instruct-2507-tput']


Full Provenance Tree

Principles Guide: principles_guide_001
  Created: 2025-11-25T18:08:35.705147
  Model: together_ai/Qwen/Qwen3-235B-A22B-Instruct-2507-tput

  Parent (Cross-Text): cross_text_synthesis_001
    Sample contributions: 5
      - sample_001 / cross_perspective_integrator
      - sample_002 / cross_perspective_integrator
      - sample_003 / cross_perspective_integrator
      ... and 2 more


## Exporting Syntheses to Filesystem

Export final syntheses to text files with YAML metadata headers for consumption by other tools or LLMs.

In [26]:
# Create outputs directory
outputs_dir = Path("outputs")
outputs_dir.mkdir(exist_ok=True)

# Export cross-text synthesis
cross_text_syntheses = store.list_syntheses('cross_text_synthesis')
if cross_text_syntheses:
    for synth_id in cross_text_syntheses:
        output_path = outputs_dir / f"{synth_id}.txt"
        store.export_synthesis(synth_id, output_path, metadata_format='yaml')
        print(f"Exported: {output_path}")

# Export principles guide
principles_guides = store.list_syntheses('principles_guide')
if principles_guides:
    for synth_id in principles_guides:
        output_path = outputs_dir / f"{synth_id}.txt"
        store.export_synthesis(synth_id, output_path, metadata_format='yaml')
        print(f"Exported: {output_path}")
        
        # Also create a special "derived_style_instructions.txt" for style_evaluation.ipynb
        if synth_id == principles_guides[-1]:  # Use latest
            instructions_path = outputs_dir / "derived_style_instructions.txt"
            store.export_synthesis(synth_id, instructions_path, metadata_format='yaml')
            print(f"Exported for style evaluation: {instructions_path}")

print(f"\nAll syntheses exported to {outputs_dir.absolute()}")

Exported: outputs/cross_text_synthesis_001.txt
Exported: outputs/principles_guide_001.txt
Exported for style evaluation: outputs/derived_style_instructions.txt

All syntheses exported to /Users/andersohrn/PycharmProjects/russell_writes/outputs
